<h1> Data Cleaning and Wrangling </h1>

In [1]:
# general purpose libraries used

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

<h2> Importing Bikeshare Station Data</h2>

In [2]:
station_df = pd.read_csv("bikeshare_stations.csv")
station_df.head()

# We can see that each row entry represents a singular location/station identified by it's Id

,Station Id,Station Name,lat,lon,capacity
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954,35
1,7001,Lower Jarvis St / The Esplanade,43.647830,-79.370698,15
2,7002,St. George St / Bloor St W,43.667333,-79.399429,19
3,7003,Madison Ave / Bloor St W,43.667158,-79.402761,15
4,7004,University Ave / Elm St,43.656518,-79.389099,11


In [3]:
# background info lookoing at the different data types and checking if there are any missing values. There are no missing values

station_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Station Id    610 non-null    int64  
 1   Station Name  610 non-null    object 
 2   lat           610 non-null    float64
 3   lon           610 non-null    float64
 4   capacity      610 non-null    int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 24.0+ KB


<h2> Importing 2017~2018 bikeshare</h2>

In [4]:
# Importing bikeshare data 2017~2018 

import os

files = os.listdir()
textfile_list = [i for i in files if 'bike_share_2017' in i or 'bike_share_2018' in i]

In [5]:
temp_data = []

for doc in textfile_list:
    temp = pd.read_csv(doc)
    temp_data.append(temp)

raw_bikeshare_2017_2018 = pd.concat(temp_data)

In [6]:
# Initial look at the data

raw_bikeshare_2017_2018.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type
0,712441,2017-01-01 05:03 (UTC),2017-01-01 05:08 (UTC),274,7006.0,Bay St / College St (East Side),7021.0,Bay St / Albert St,Member
1,712442,2017-01-01 05:03 (UTC),2017-01-01 05:12 (UTC),538,7046.0,Niagara St / Richmond St W,7147.0,King St W / Fraser Ave,Member
2,712443,2017-01-01 05:05 (UTC),2017-01-01 05:22 (UTC),992,7048.0,Front St / Yonge St (Hockey Hall of Fame),7089.0,Church St / Wood St,Member
3,712444,2017-01-01 05:09 (UTC),2017-01-01 05:26 (UTC),1005,7177.0,East Liberty St / Pirandello St,7202.0,Queen St W / York St (City Hall),Member
4,712445,2017-01-01 05:14 (UTC),2017-01-01 05:25 (UTC),645,7203.0,Bathurst St / Queens Quay W,7010.0,King St W / Spadina Ave,Member


In [7]:
# Background information

raw_bikeshare_2017_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3415324 entries, 0 to 255000
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   trip_id                int64  
 1   trip_start_time        object 
 2   trip_stop_time         object 
 3   trip_duration_seconds  int64  
 4   from_station_id        float64
 5   from_station_name      object 
 6   to_station_id          float64
 7   to_station_name        object 
 8   user_type              object 
dtypes: float64(2), int64(2), object(5)
memory usage: 260.6+ MB


In [8]:
# remove any duplicate trips based on trip ID if there are any
# Assumption is that we will only keep the first occurence of the trip with the same trip ID

bikeshare_2017_2018 = raw_bikeshare_2017_2018.drop_duplicates(subset='trip_id', keep='first')
len(bikeshare_2017_2018)

# Seems like there are no duplicate entries, this is good

3415324

In [9]:
# Find missing data if any

bikeshare_2017_2018.count()

# We see that there are some missing data in from_station_id, to_station_id, from_station_name, and to_station_name. Let's explore what they look like

trip_id                  3415324
trip_start_time          3415324
trip_stop_time           3415323
trip_duration_seconds    3415324
from_station_id          2388431
from_station_name        3415324
to_station_id            2388431
to_station_name          3415323
user_type                3415324
dtype: int64

In [10]:
# We will remove the data with missing 'trip_stop_time' and 'to_station_name' since without these essential information, the observation would not be useful 
# in understanding the geographical features such as distance travelled and time

bikeshare_2017_2018 = bikeshare_2017_2018[(bikeshare_2017_2018['trip_stop_time'].notnull())| (bikeshare_2017_2018['to_station_name'].notnull())]

In [11]:
null_data = bikeshare_2017_2018[bikeshare_2017_2018.isnull().any(axis=1)]

null_data.head()

# seems like some stations are missing data, we can get the missing information from the "bikeshare_stations.csv"

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type
0,1971685,2017-10-01 05:00 (UTC),2017-10-01 05:14 (UTC),849,NaN,Queen St W / James St,NaN,Taddle Creek Park,Member
1,1971686,2017-10-01 05:00 (UTC),2017-10-01 05:00 (UTC),26,NaN,Stewart St / Bathurst St - SMART,NaN,Stewart St / Bathurst St - SMART,Casual
2,1971687,2017-10-01 05:00 (UTC),2017-10-01 05:22 (UTC),1310,NaN,Hayter St / Laplante Ave,NaN,Queen St W / Portland St,Member
3,1971688,2017-10-01 05:00 (UTC),2017-10-01 05:11 (UTC),631,NaN,Beverly St / College St,NaN,Beverly St / Dundas St W,Member
4,1971689,2017-10-01 05:00 (UTC),2017-10-01 05:20 (UTC),1198,NaN,Thompson St / Broadview Ave - SMART,NaN,Thompson St / Broadview Ave - SMART,Member


In [12]:
# merge bikeshare stations with 2017~2018 bikeshare data to find missing station id. Start with from_station_id

merged_2017_2018 = pd.merge(bikeshare_2017_2018, station_df, how='left', left_on='from_station_name', right_on='Station Name')
merged_2017_2018['from_station_id'] = merged_2017_2018['from_station_id'].fillna(merged_2017_2018['Station Id'])

merged_2017_2018 = merged_2017_2018[merged_2017_2018.columns[:-5]]

In [13]:
# merge bikeshare stations with 2017~2018 bikeshare data to find missing station id. End with to_station_id

merged_2017_2018 = pd.merge(merged_2017_2018 , station_df, how='left', left_on='to_station_name', right_on='Station Name')
merged_2017_2018['to_station_id'] = merged_2017_2018['to_station_id'].fillna(merged_2017_2018['Station Id'])

merged_2017_2018 = merged_2017_2018[merged_2017_2018.columns[:-5]]

In [14]:
# Let's check for missing data now, there seems to be some improvement but we should look into it further  

len(merged_2017_2018) - merged_2017_2018.count()

trip_id                       0
trip_start_time               0
trip_stop_time                0
trip_duration_seconds         0
from_station_id          207212
from_station_name             0
to_station_id            207016
to_station_name               0
user_type                     0
dtype: int64

In [15]:
# Here we will create a function to understand the percetage of missing data within the selected column
# This could be useful in the future for determining whether to keep going or not

def missing_percent(data, column): 
    percent = data[column].isna().sum()/len(data)
    return print('{0:.2%}'.format(percent))

In [16]:
# 6% of 'from_station_id' is missing values
missing_percent(merged_2017_2018,'from_station_id')

6.07%


In [17]:
# 6% of 'from_station_id' is missing values
missing_percent(merged_2017_2018,'to_station_id')

6.06%


In [18]:
# Let's find out the exact station names that could not be identified

null_data = merged_2017_2018[merged_2017_2018.isnull().any(axis=1)]
null_data.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type
41420,1971688,2017-10-01 05:00 (UTC),2017-10-01 05:11 (UTC),631,NaN,Beverly St / College St,NaN,Beverly St / Dundas St W,Member
41422,1971690,2017-10-01 05:01 (UTC),2017-10-01 05:07 (UTC),396,7024.0,Dundonald St / Church St,NaN,Victoria St / Gould St (Ryerson University),Member
41427,1971695,2017-10-01 05:01 (UTC),2017-10-01 05:30 (UTC),1698,NaN,University Ave / College St,7021.0,Bay St / Albert St,Casual
41431,1971699,2017-10-01 05:03 (UTC),2017-10-01 05:12 (UTC),534,7275.0,Queen St W / James St,NaN,25 York St (ACC/Union Station South),Member
41433,1971701,2017-10-01 05:03 (UTC),2017-10-01 05:15 (UTC),692,7192.0,Harbord St / Clinton St,NaN,University Ave / College St,Member


In [19]:
null_stations = []

for index, row in null_data.iterrows():
    if pd.isna(row['from_station_id']):
        null_stations.append(row['from_station_name'])
    elif pd.isna(row['to_station_id']):
        null_stations.append(row['to_station_name'])

null_stations = set(null_stations)

In [20]:
# Looking at the bikeshare stations, we find that the station Id indeed exists.
# We need to replace the existing names with the correct ones in the bikshare_stations.csv

null_stations

{'25 York St (ACC/Union Station South)',
 '80 Clinton St (North of College)',
 'Base Station',
 'Bathurst St / Lennox St',
 'Bathurst St / Queens Quay W',
 'Bathurst Subway Station',
 'Bay St / Bloor St W',
 'Bay St / Davenport Rd',
 'Beverly  St / Dundas St W',
 'Beverly St / College St',
 'Bloor St / Brunswick Ave',
 'Bloor St W / Christie St',
 'Bremner Blvd / Spadina Ave',
 'Bridgeman Ave / Bathurst St',
 'Bridgeman Ave / Kendal Ave',
 'Chester Station',
 'Davenport Rd / Bathurst St',
 'Davenport Rd / Christie St',
 'Dockside Dr / Queens Quay E (Sugar Beach)',
 'Dovercourt Rd / Harrison St - SMART',
 'Dufferin St / Bloor St',
 'Essex St / Christie St',
 'Euclid Ave / Bloor St W',
 'Fort York  Blvd / Capreol Crt',
 'Fringe Next Stage - 7219',
 'Lake Shore Blvd W / Ontario Dr(Ontario Place)',
 'Lakeshore Blvd W / Ellis Ave',
 'Lakeshore Blvd W / The Boulevard Club',
 'Lansdowne Subway Green P',
 'Macpherson Ave / Spadina Rd',
 'Margueretta St / College St',
 'Michael Sweet Ave / St. 

In [21]:
# replace station names with the correct ones found in the bikeshare_stations.csv
# I will not replace stations on the same intersection with multiple stations since I don't know which station is the correct one
# Ex: 'University Ave / College St (East)' or 'University Ave / College St (West)' when we are only given 'University Ave / College St'

def replace_name_2017_2018(a,b):
    bikeshare_2017_2018.from_station_name.replace(a,b,inplace=True)
    bikeshare_2017_2018.to_station_name.replace(a,b,inplace=True)

replace_name_2017_2018('25 York St (ACC/Union Station South)','25 York St – Union Station South')
replace_name_2017_2018('80 Clinton St (North of College)','80 Clinton St (North of College) - SMART')
replace_name_2017_2018('Bathurst St / Lennox St','Bathurst St / Lennox St - SMART')
replace_name_2017_2018('Bathurst St / Queens Quay W','Bathurst St/Queens Quay(Billy Bishop Airport)')
replace_name_2017_2018('Bathurst Subway Station','Bathurst Subway Station - SMART')
replace_name_2017_2018('Beverly  St / Dundas St W','Beverley  St / Dundas St W')
replace_name_2017_2018('Bloor St W / Christie St','Bloor St W / Christie St - SMART')
replace_name_2017_2018('Bridgeman Ave / Bathurst St','Bridgeman Ave / Bathurst St - SMART')
replace_name_2017_2018('Bridgeman Ave / Kendal Ave','Bridgeman Ave / Kendal Ave - SMART')
replace_name_2017_2018('Chester Station','Chester Station (Green P)')
replace_name_2017_2018('Davenport Rd / Bathurst St','Davenport Rd / Bathurst St - SMART')
replace_name_2017_2018('Davenport Rd / Christie St','Davenport Rd / Christie St - SMART')
replace_name_2017_2018('Dovercourt Rd / Harrison St - SMART','Dovercourt Rd / Harrison St (Green P) - SMART')
replace_name_2017_2018('Dufferin St / Bloor St','Dufferin St / Bloor St- SMART')
replace_name_2017_2018('Essex St / Christie St','Essex St / Christie St - SMART')
replace_name_2017_2018('Fort York  Blvd / Capreol Crt','Fort York  Blvd / Capreol Ct')
replace_name_2017_2018('Lake Shore Blvd W / Ontario Dr(Ontario Place)','Lake Shore Blvd W / Ontario Dr')
replace_name_2017_2018('Lakeshore Blvd W / Ellis Ave','Lake Shore Blvd W / Ellis Ave')
replace_name_2017_2018('Lakeshore Blvd W / The Boulevard Club','Lake Shore Blvd W / The Boulevard Club') 
replace_name_2017_2018('Lansdowne Subway Green P','Lansdowne Subway Station')
replace_name_2017_2018('Macpherson Ave / Spadina Rd','Macpherson Ave / Spadina Rd - SMART')
replace_name_2017_2018('Ossington Ave / Bloor St W','Ossington Ave / Bloor St W - SMART') 
replace_name_2017_2018('Palmerston Ave / Vermont Ave','Palmerston Ave / Vermont Ave - SMART') 
replace_name_2017_2018('Queen St E / Berkely St','Queen St E / Berkeley St') 
replace_name_2017_2018('Salem Ave / Bloor St W','Salem Ave / Bloor St W - SMART')
replace_name_2017_2018('Seaton St / Dundas St E','Seaton St / Dundas St E - SMART') 
replace_name_2017_2018('Sterling Rd / Dundas St W','Sterling Rd / Dundas St W - SMART') 
replace_name_2017_2018('Summerhill Ave / MacLennan Ave - SMART','Summerhill Ave / Maclennan Ave') 
replace_name_2017_2018('University Ave / King St W','University Ave / King St W - SMART')
replace_name_2017_2018('Woodbine Subway Green P (Cedarvale Ave)','Woodbine Subway Green P SMART') 
replace_name_2017_2018('Yarmouth Rd / Christie St','Yarmouth Rd / Christie St - SMART') 
replace_name_2017_2018('York St / King St W - SMART','King St W / York St')

In [43]:
# merge bikeshare stations with 2017~2018 bikeshare data again to import station ID
# We will also create a merge time that is rounded to the hour to merge with the weather data later

merged_2017_2018 = pd.merge(bikeshare_2017_2018, station_df, how='left', left_on='from_station_name', right_on='Station Name')
merged_2017_2018['from_station_id'] = merged_2017_2018['from_station_id'].fillna(merged_2017_2018['Station Id'])

merged_2017_2018 = merged_2017_2018[merged_2017_2018.columns[:-5]]

merged_2017_2018 = pd.merge(merged_2017_2018 , station_df, how='left', left_on='to_station_name', right_on='Station Name')
merged_2017_2018['to_station_id'] = merged_2017_2018['to_station_id'].fillna(merged_2017_2018['Station Id'])

merged_2017_2018 = merged_2017_2018[merged_2017_2018.columns[:-5]]

In [44]:
# we've decreased the missing data percentage from 6% to 3%

missing_percent(merged_2017_2018,'from_station_id')

3.46%


In [45]:
# we've decreased the missing data percentage from 6% to 4%

missing_percent(merged_2017_2018,'to_station_id')

3.55%


In [46]:
len(merged_2017_2018) - merged_2017_2018.count()

trip_id                       0
trip_start_time               0
trip_stop_time                0
trip_duration_seconds         0
from_station_id          118277
from_station_name             0
to_station_id            121244
to_station_name               0
user_type                     0
dtype: int64

In [47]:
# We will drop the remaining rows with missing data since we are unable to determine the exact station id from the 'bikeshare_stations.csv' or the particular station does not exist in 'bikeshare_stations.csv'

no_missing_2017_2018 = merged_2017_2018.dropna()

"{:.2%}".format(len(no_missing_2017_2018)/len(raw_bikeshare_2017_2018))

# In the end, we will be using 93.52% of the original raw bikeshare data between 2017~2018

'93.52%'

In [48]:
# Convert to Da
# str[:-5] will remove the (UTC) that will interfere with the conversion

import datetime
pd.options.mode.chained_assignment = None  # default='warn'

no_missing_2017_2018['trip_start_time'] = pd.to_datetime(no_missing_2017_2018['trip_start_time'].str[:-5]).dt.tz_localize('UTC')
no_missing_2017_2018['Merge Time'] = no_missing_2017_2018['trip_start_time'].dt.round('60min').dt.tz_convert('US/Eastern')

no_missing_2017_2018['trip_start_time'] = no_missing_2017_2018['trip_start_time'].dt.tz_convert('US/Eastern')

no_missing_2017_2018['trip_stop_time'] = pd.to_datetime(no_missing_2017_2018['trip_stop_time'].str[:-5]).dt.tz_localize('UTC')
no_missing_2017_2018['trip_stop_time'] = no_missing_2017_2018['trip_stop_time'].dt.tz_convert('US/Eastern')

In [49]:
# station Id to int from float

no_missing_2017_2018['from_station_id'] = no_missing_2017_2018['from_station_id'].astype('int')
no_missing_2017_2018['to_station_id'] = no_missing_2017_2018['to_station_id'].astype('int')

In [50]:
# The final step is to convert the column names into those of the 2019~2020 data so that they can be concatanated later on 

column_namechange = {
    'trip_id': 'Trip Id',
    'trip_start_time': 'Start Time',
    'trip_stop_time': 'End Time',
    'trip_duration_seconds': 'Trip Duration',
    'from_station_id': 'Start Station Id',
    'from_station_name': 'Start Station Name',
    'to_station_id': 'End Station Id',
    'to_station_name': 'End Station Name',
    'user_type': 'User Type'
}

no_missing_2017_2018 = no_missing_2017_2018.rename(columns=column_namechange)

In [51]:
# We have finished our cleaning for now, time to move on to bikeshare 2019~2020 data

no_missing_2017_2018.head()

,Trip Id,Start Time,End Time,Trip Duration,Start Station Id,Start Station Name,End Station Id,End Station Name,User Type,Merge Time
0,712441,2017-01-01 00:03:00-05:00,2017-01-01 00:08:00-05:00,274,7006,Bay St / College St (East Side),7021,Bay St / Albert St,Member,2017-01-01 00:00:00-05:00
1,712442,2017-01-01 00:03:00-05:00,2017-01-01 00:12:00-05:00,538,7046,Niagara St / Richmond St W,7147,King St W / Fraser Ave,Member,2017-01-01 00:00:00-05:00
2,712443,2017-01-01 00:05:00-05:00,2017-01-01 00:22:00-05:00,992,7048,Front St / Yonge St (Hockey Hall of Fame),7089,Church St / Wood St,Member,2017-01-01 00:00:00-05:00
3,712444,2017-01-01 00:09:00-05:00,2017-01-01 00:26:00-05:00,1005,7177,East Liberty St / Pirandello St,7202,Queen St W / York St (City Hall),Member,2017-01-01 00:00:00-05:00
4,712445,2017-01-01 00:14:00-05:00,2017-01-01 00:25:00-05:00,645,7203,Bathurst St/Queens Quay(Billy Bishop Airport),7010,King St W / Spadina Ave,Member,2017-01-01 00:00:00-05:00


<h2> Importing 2019~2020 data </h2>

In [71]:
files = os.listdir()
textfile_list = [i for i in files if "bike_share_2019" in i or "bike_share_2020" in i]

In [72]:
temp_data = []

for doc in textfile_list:
    temp = pd.read_csv(doc)
    temp_data.append(temp)

raw_bikeshare_2019_2020  = pd.concat(temp_data)

In [73]:
# Initial look at the data

raw_bikeshare_2019_2020.head()

,Trip Id,Subscription Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
0,4581278,199751,1547,7021,01/01/2019 00:08:00 (EST),Bay St / Albert St,7233.0,01/01/2019 00:33:00 (EST),King / Cowan Ave - SMART,1296,Annual Member
1,4581279,294730,1112,7160,01/01/2019 00:10:00 (EST),King St W / Tecumseth St,7051.0,01/01/2019 00:29:00 (EST),Wellesley St E / Yonge St (Green P),2947,Annual Member
2,4581280,197252,589,7055,01/01/2019 00:15:00 (EST),Jarvis St / Carlton St,7013.0,01/01/2019 00:25:00 (EST),Scott St / The Esplanade,2293,Annual Member
3,4581281,171700,259,7012,01/01/2019 00:16:00 (EST),Elizabeth St / Edward St (Bus Terminal),7235.0,01/01/2019 00:20:00 (EST),Bay St / College St (West Side) - SMART,283,Annual Member
4,4581282,306314,281,7041,01/01/2019 00:19:00 (EST),Edward St / Yonge St,7257.0,01/01/2019 00:24:00 (EST),Dundas St W / St. Patrick St,1799,Annual Member


In [74]:
# Background information

raw_bikeshare_2019_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5052221 entries, 0 to 440661
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Trip Id             int64  
 1   Subscription Id     int64  
 2   Trip  Duration      int64  
 3   Start Station Id    int64  
 4   Start Time          object 
 5   Start Station Name  object 
 6   End Station Id      float64
 7   End Time            object 
 8   End Station Name    object 
 9   Bike Id             int64  
 10  User Type           object 
dtypes: float64(1), int64(5), object(5)
memory usage: 462.5+ MB


In [75]:
# remove any duplicate trips based on trip ID if there are any
# Assumption is that we will only keep the first occurence of the trip with the same trip ID

bikeshare_2019_2020 = raw_bikeshare_2019_2020.drop_duplicates(subset='Trip Id', keep='first')
len(bikeshare_2019_2020)

# Seems like there are no duplicate entries, this is good

5052221

In [76]:
# Find missing data if any

len(bikeshare_2019_2020) - bikeshare_2019_2020.count()

# We see that there are some missing data in 'Start Station Name', 'End Station Id', and 'End Station Name'. Let's explore what they look like

Trip Id                  0
Subscription Id          0
Trip  Duration           0
Start Station Id         0
Start Time               0
Start Station Name     164
End Station Id        1266
End Time                 0
End Station Name      1408
Bike Id                  0
User Type                0
dtype: int64

In [77]:
# First, we will drop any rows with both 'End Station Name' and 'End Station Id' as missing values. Without both these values, it would be impossible to tell which station did the biker disembark at

inds = bikeshare_2019_2020[["End Station Id", "End Station Name"]].isnull().all(axis=1) 
bikeshare_2019_2020 = bikeshare_2019_2020.loc[~inds, :]

len(bikeshare_2019_2020) - bikeshare_2019_2020.count()

# Now we only have 'Start Station Name' and 'End Station Name' with missing values

Trip Id                 0
Subscription Id         0
Trip  Duration          0
Start Station Id        0
Start Time              0
Start Station Name    164
End Station Id          0
End Time                0
End Station Name      142
Bike Id                 0
User Type               0
dtype: int64

In [78]:
# merge bikeshare stations with 2019~2020 bikeshare data to find missing Start Station Name.
# We will also create a merge time that is rounded to the hour to merge with the weather data later

merged_2019_2020 = pd.merge(bikeshare_2019_2020, station_df, how='left', left_on='Start Station Id', right_on='Station Id')
merged_2019_2020['Start Station Name'] = merged_2019_2020['Start Station Name'].fillna(merged_2019_2020['Station Name'])

merged_2019_2020 = merged_2019_2020[merged_2019_2020.columns[:-5]]

len(bikeshare_2019_2020) - merged_2019_2020.count()

# lastly we have 'End Station Name' with missing values

Trip Id                 0
Subscription Id         0
Trip  Duration          0
Start Station Id        0
Start Time              0
Start Station Name      0
End Station Id          0
End Time                0
End Station Name      142
Bike Id                 0
User Type               0
dtype: int64

In [79]:
# merge bikeshare stations with 2019~2020 bikeshare data to find missing End Station Name.

merged_2019_2020 = pd.merge(merged_2019_2020, station_df, how='left', left_on='End Station Id', right_on='Station Id')
merged_2019_2020['End Station Name'] = merged_2019_2020['End Station Name'].fillna(merged_2019_2020['Station Name'])

merged_2019_2020 = merged_2019_2020[merged_2019_2020.columns[:-5]]

len(bikeshare_2019_2020) - merged_2019_2020.count()

# We have found all the missing values left in the dataset

Trip Id               0
Subscription Id       0
Trip  Duration        0
Start Station Id      0
Start Time            0
Start Station Name    0
End Station Id        0
End Time              0
End Station Name      0
Bike Id               0
User Type             0
dtype: int64

In [80]:
no_missing_2019_2020 = merged_2019_2020

"{:.2%}".format(len(no_missing_2019_2020)/len(raw_bikeshare_2019_2020))

# In the end, we will be using 99.97% of the original raw bikeshare data between 2017~2018

'99.97%'

In [81]:
# Timestamp to EST
# str[:-5] will remove the (EST) that will interfere with the conversion

no_missing_2019_2020['Start Time'] = pd.to_datetime(no_missing_2019_2020['Start Time'].str[:-5]).dt.tz_localize('EST')
no_missing_2019_2020['Merge Time'] = no_missing_2019_2020['Start Time'].dt.round('60min')

no_missing_2019_2020['End Time'] = pd.to_datetime(no_missing_2019_2020['End Time'].str[:-5]).dt.tz_localize('EST')

In [82]:
# As noted when our team recieved this project, there are 2 extra columns of 'Subscription Id' and 'Bike Id' in the 2019~2020 dataset that is not present in the 2017~2018 dataset. We will calculate the percentage of the total dataset from 2017~2020 that is missing 'Subscription Id' and 'Bike Id' to determine if these columns should be kept or not

extra_columns_percent = 1 - len(no_missing_2019_2020)/(len(no_missing_2019_2020) + len(no_missing_2017_2018))
print("A total of " + "{:.2%}".format(extra_columns_percent) + " rows that are missing 'Subscription Id' and 'Bike Id'")

A total of 38.74% rows that are missing 'Subscription Id' and 'Bike Id'


In [83]:
# Since we have close to 40% of our data that does not have 'Subscription Id' and 'Bike Id', we will drop these 2 columns

no_missing_2019_2020 = no_missing_2019_2020.drop(['Subscription Id', 'Bike Id'], axis = 1) 
no_missing_2019_2020.head()

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,User Type,Merge Time
0,4581278,1547,7021,2019-01-01 00:08:00-05:00,Bay St / Albert St,7233.0,2019-01-01 00:33:00-05:00,King / Cowan Ave - SMART,Annual Member,2019-01-01 00:00:00-05:00
1,4581279,1112,7160,2019-01-01 00:10:00-05:00,King St W / Tecumseth St,7051.0,2019-01-01 00:29:00-05:00,Wellesley St E / Yonge St (Green P),Annual Member,2019-01-01 00:00:00-05:00
2,4581280,589,7055,2019-01-01 00:15:00-05:00,Jarvis St / Carlton St,7013.0,2019-01-01 00:25:00-05:00,Scott St / The Esplanade,Annual Member,2019-01-01 00:00:00-05:00
3,4581281,259,7012,2019-01-01 00:16:00-05:00,Elizabeth St / Edward St (Bus Terminal),7235.0,2019-01-01 00:20:00-05:00,Bay St / College St (West Side) - SMART,Annual Member,2019-01-01 00:00:00-05:00
4,4581282,281,7041,2019-01-01 00:19:00-05:00,Edward St / Yonge St,7257.0,2019-01-01 00:24:00-05:00,Dundas St W / St. Patrick St,Annual Member,2019-01-01 00:00:00-05:00


In [84]:
# End station Id to int from float

no_missing_2019_2020['End Station Id'] = no_missing_2019_2020['End Station Id'].astype('int')

In [85]:
# The last step before combining the 2 datasets tigether us to change the order of columns to match that of 2017~2018

col = list(no_missing_2019_2020.columns.values)
col

['Trip Id',
 'Trip  Duration',
 'Start Station Id',
 'Start Time',
 'Start Station Name',
 'End Station Id',
 'End Time',
 'End Station Name',
 'User Type',
 'Merge Time']

In [86]:
# There seems to be an extra space in 'Trip Duration' that should be removed
# Let's remap the columns to match the 2017~2018 data

no_missing_2019_2020 = no_missing_2019_2020.rename(columns={'Trip  Duration':'Trip Duration'})

no_missing_2019_2020 = no_missing_2019_2020[['Trip Id', 'Start Time', 'End Time', 'Trip Duration', 'Start Station Id', 'Start Station Name', 'End Station Id', 'End Station Name', 'User Type', 'Merge Time']]

no_missing_2019_2020.head()

,Trip Id,Start Time,End Time,Trip Duration,Start Station Id,Start Station Name,End Station Id,End Station Name,User Type,Merge Time
0,4581278,2019-01-01 00:08:00-05:00,2019-01-01 00:33:00-05:00,1547,7021,Bay St / Albert St,7233,King / Cowan Ave - SMART,Annual Member,2019-01-01 00:00:00-05:00
1,4581279,2019-01-01 00:10:00-05:00,2019-01-01 00:29:00-05:00,1112,7160,King St W / Tecumseth St,7051,Wellesley St E / Yonge St (Green P),Annual Member,2019-01-01 00:00:00-05:00
2,4581280,2019-01-01 00:15:00-05:00,2019-01-01 00:25:00-05:00,589,7055,Jarvis St / Carlton St,7013,Scott St / The Esplanade,Annual Member,2019-01-01 00:00:00-05:00
3,4581281,2019-01-01 00:16:00-05:00,2019-01-01 00:20:00-05:00,259,7012,Elizabeth St / Edward St (Bus Terminal),7235,Bay St / College St (West Side) - SMART,Annual Member,2019-01-01 00:00:00-05:00
4,4581282,2019-01-01 00:19:00-05:00,2019-01-01 00:24:00-05:00,281,7041,Edward St / Yonge St,7257,Dundas St W / St. Patrick St,Annual Member,2019-01-01 00:00:00-05:00


<h2> Combining Bikeshare 2017~2018 with 2019~2020 </h2>

In [87]:
bikeshare_df = no_missing_2017_2018.append(no_missing_2019_2020, ignore_index=True)

In [88]:
# The last step before cleanig is complete for bikeshare data is to remove the overlapping names in User Type

bikeshare_df['User Type'].unique()

array(['Member', 'Casual', 'Annual Member', 'Casual Member'], dtype=object)

In [89]:
# There should be 2 only of Annual Member and Casual Member

bikeshare_df['User Type'] = bikeshare_df['User Type'].replace({'Member': 'Annual Member', 'Casual': 'Casual Member'})
bikeshare_df['User Type'].unique()

array(['Annual Member', 'Casual Member'], dtype=object)

In [90]:
# Check for outliers

bikeshare_df.describe()

# Seems like there are trips that are 0 seconds which does not seem plausible. The max trip is roughly 143.5 days which is also not reasonable too 

,Trip Id,Trip Duration,Start Station Id,End Station Id
count,8.245040e+06,8.245040e+06,8.245040e+06,8.245040e+06
mean,5.603315e+06,1.070320e+03,7.178719e+03,7.177912e+03
std,2.733524e+06,1.161226e+04,1.446954e+02,1.443167e+02
min,7.123820e+05,0.000000e+00,7.000000e+03,7.000000e+03
25%,3.304106e+06,4.410000e+02,7.051000e+03,7.051000e+03
50%,5.633330e+06,7.220000e+02,7.158000e+03,7.157000e+03
75%,7.980158e+06,1.139000e+03,7.270000e+03,7.269000e+03
max,1.029388e+07,1.240378e+07,7.667000e+03,7.667000e+03


In [91]:
# We will create a function that only keep trips that are at least 60 seconds. Afterwards we will remove any outliers that are less than Q1 - 1.5 * IQR and greater than Q3 + 1.5 * IQR

def duration_outlier(data, column):
    data = data[[i for i in data[column] >= 60]]

    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr

    data = data[(data[column] >= lower) & (data[column] <= upper)]

    return data
    

bikeshare_df = duration_outlier(bikeshare_df, 'Trip Duration')
bikeshare_df.describe()


,Trip Id,Trip Duration,Start Station Id,End Station Id
count,7.835134e+06,7.835134e+06,7.835134e+06,7.835134e+06
mean,5.613399e+06,7.865584e+02,7.177588e+03,7.176582e+03
std,2.727796e+06,4.447951e+02,1.441453e+02,1.437636e+02
min,7.123820e+05,6.000000e+01,7.000000e+03,7.000000e+03
25%,3.331983e+06,4.340000e+02,7.051000e+03,7.050000e+03
50%,5.645072e+06,6.970000e+02,7.155000e+03,7.154000e+03
75%,7.965562e+06,1.072000e+03,7.269000e+03,7.267000e+03
max,1.029388e+07,2.190000e+03,7.667000e+03,7.667000e+03


In [92]:
# The new minimum trip time is 60 seconds and the max trip is 36.5 minutes

bikeshare_df['Trip Duration'].max()/60

36.5

In [93]:
# Cleaned Bike Data

clean_bikeshare = bikeshare_df
clean_bikeshare.head()

,Trip Id,Start Time,End Time,Trip Duration,Start Station Id,Start Station Name,End Station Id,End Station Name,User Type,Merge Time
0,712441,2017-01-01 00:03:00-05:00,2017-01-01 00:08:00-05:00,274,7006,Bay St / College St (East Side),7021,Bay St / Albert St,Annual Member,2017-01-01 00:00:00-05:00
1,712442,2017-01-01 00:03:00-05:00,2017-01-01 00:12:00-05:00,538,7046,Niagara St / Richmond St W,7147,King St W / Fraser Ave,Annual Member,2017-01-01 00:00:00-05:00
2,712443,2017-01-01 00:05:00-05:00,2017-01-01 00:22:00-05:00,992,7048,Front St / Yonge St (Hockey Hall of Fame),7089,Church St / Wood St,Annual Member,2017-01-01 00:00:00-05:00
3,712444,2017-01-01 00:09:00-05:00,2017-01-01 00:26:00-05:00,1005,7177,East Liberty St / Pirandello St,7202,Queen St W / York St (City Hall),Annual Member,2017-01-01 00:00:00-05:00
4,712445,2017-01-01 00:14:00-05:00,2017-01-01 00:25:00-05:00,645,7203,Bathurst St/Queens Quay(Billy Bishop Airport),7010,King St W / Spadina Ave,Annual Member,2017-01-01 00:00:00-05:00


In [94]:
# Percentage of total data used
"{:.2%}".format(len(clean_bikeshare)/(len(raw_bikeshare_2017_2018) + len(raw_bikeshare_2019_2020)))

# In the end, we will be using 92.53% of the original raw bikeshare data between 2017~2020

'92.53%'

<h2> Weather Data </h2>

In [95]:
files = os.listdir()
textfile_list = [i for i in files if "climate" in i]

In [96]:
temp_data = []

for doc in textfile_list:
    temp = pd.read_csv(doc)
    temp_data.append(temp)

raw_weather_df  = pd.concat(temp_data)
raw_weather_df.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
0,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017-01-01 00:00,2017,1,1,00:00,1.5,...,NaN,16.1,NaN,99.81,NaN,NaN,NaN,NaN,NaN,NaN
1,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017-01-01 01:00,2017,1,1,01:00,1.5,...,NaN,16.1,NaN,100.01,NaN,NaN,NaN,NaN,NaN,NaN
2,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017-01-01 02:00,2017,1,1,02:00,1.0,...,NaN,16.1,NaN,100.14,NaN,NaN,NaN,NaN,NaN,NaN
3,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017-01-01 03:00,2017,1,1,03:00,1.2,...,NaN,16.1,NaN,100.32,NaN,NaN,NaN,NaN,NaN,NaN
4,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017-01-01 04:00,2017,1,1,04:00,1.3,...,NaN,16.1,NaN,100.48,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
# background information

raw_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35064 entries, 0 to 743
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Longitude (x)        35064 non-null  float64
 1   Latitude (y)         35064 non-null  float64
 2   Station Name         35064 non-null  object 
 3   Climate ID           35064 non-null  int64  
 4   Date/Time            35064 non-null  object 
 5   Year                 35064 non-null  int64  
 6   Month                35064 non-null  int64  
 7   Day                  35064 non-null  int64  
 8   Time                 35064 non-null  object 
 9   Temp (°C)            34605 non-null  float64
 10  Temp Flag            89 non-null     object 
 11  Dew Point Temp (°C)  34556 non-null  float64
 12  Dew Point Temp Flag  139 non-null    object 
 13  Rel Hum (%)          34564 non-null  float64
 14  Rel Hum Flag         131 non-null    object 
 15  Wind Dir (10s deg)   32679 non-null  f

In [98]:
# Let's check for missing volume

missing_weather = raw_weather_df.copy()
missing_weather = missing_weather.T
missing_weather["count"] = missing_weather.isnull().sum(axis=1)

weather_data_missing = pd.DataFrame(missing_weather["count"])

weather_data_missing

,count
Longitude (x),0
Latitude (y),0
Station Name,0
Climate ID,0
Date/Time,0
Year,0
Month,0
Day,0
Time,0
Temp (°C),459


In [99]:
# After some research on what 'Flags' meant in the weather context, it was found to be the Flag Index that displays the status of the associated measurement
# Ex: 'Dew Point Temp Flag' would be the 'Flag' for 'Dew Point Temp'
# Reference: https://cran.r-project.org/web/packages/weathercan/vignettes/flags.html

raw_weather_df['Dew Point Temp Flag'].unique()

# 'M' in this case means 'Missing'

array([nan, 'M'], dtype=object)

In [100]:
# Since we know that there are 35064 entries in total, we can drop the columns that have all their data as Nan
# These columns include 'Wind Chill Flag' and 'Hmdx Flag'
# We will also drop any columns with "Flag" since no description was giving for what they meant. We also have the associated measurement columns so the Flags would be unecessary in our analysis

weather_data = raw_weather_df.drop(['Temp Flag', 'Dew Point Temp Flag', 'Rel Hum Flag', 'Wind Dir Flag', 'Wind Spd Flag', 'Visibility Flag', 'Stn Press Flag', 'Hmdx Flag', 'Wind Chill Flag'], axis=1)

In [101]:
# How many missing data left

len(weather_data) - weather_data.count()

Longitude (x)              0
Latitude (y)               0
Station Name               0
Climate ID                 0
Date/Time                  0
Year                       0
Month                      0
Day                        0
Time                       0
Temp (°C)                459
Dew Point Temp (°C)      508
Rel Hum (%)              500
Wind Dir (10s deg)      2385
Wind Spd (km/h)          384
Visibility (km)          391
Stn Press (kPa)          462
Hmdx                   29397
Wind Chill             28998
Weather                29547
dtype: int64

In [102]:
# Looking at the weather, it seems that NaN would represent clear days, there may be other columns where NaN has a meaning
# The full exploration of NaN and their inclusion in our model analysis will be included in the future notebooks of Exploratory Data Analysis

weather_data['Weather'].unique()

array([nan, 'Fog', 'Rain,Fog', 'Rain', 'Snow', 'Moderate Rain',
       'Moderate Rain,Fog', 'Haze', 'Rain,Snow', 'Freezing Rain,Fog',
       'Snow,Blowing Snow', 'Heavy Snow', 'Moderate Snow',
       'Haze,Blowing Snow', 'Heavy Rain,Fog', 'Thunderstorms,Rain,Fog',
       'Freezing Rain,Snow', 'Freezing Rain', 'Thunderstorms,Rain',
       'Thunderstorms,Moderate Rain,Fog', 'Thunderstorms,Moderate Rain',
       'Thunderstorms', 'Thunderstorms,Heavy Rain,Fog',
       'Thunderstorms,Heavy Rain', 'Thunderstorms,Fog'], dtype=object)

In [103]:
# Let's make the index be the 'Date/Time' and convert it to Eastern Time so that we can easily merge this dataset with the bikeshare data later

weather_data.index = pd.DatetimeIndex(weather_data["Date/Time"])
weather_data = weather_data.drop(columns=["Date/Time"], axis=1)
weather_data.index = weather_data.index.tz_localize(tz='EST')

,Longitude (x),Latitude (y),Station Name,Climate ID,Year,Month,Day,Time,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Hmdx,Wind Chill,Weather
Date/Time,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00-05:00,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017,1,1,00:00,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN
2017-01-01 01:00:00-05:00,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017,1,1,01:00,1.5,-3.9,67.0,27.0,35.0,16.1,100.01,NaN,NaN,NaN
2017-01-01 02:00:00-05:00,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017,1,1,02:00,1.0,-4.3,68.0,26.0,32.0,16.1,100.14,NaN,NaN,NaN
2017-01-01 03:00:00-05:00,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017,1,1,03:00,1.2,-4.3,67.0,26.0,37.0,16.1,100.32,NaN,NaN,NaN
2017-01-01 04:00:00-05:00,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017,1,1,04:00,1.3,-4.4,66.0,26.0,28.0,16.1,100.48,NaN,NaN,NaN


In [ ]:
clean_weather = weather_data
# View DataFrame
clean_weather.head()

<h2> Merge Bikeshare and Weather </h2>

In [138]:
# We will merge the weather data using the 'Merge Time' of each bikershare trip with the 'Date/Time' index of the weather data. 

clean_bikeshare['Merge Time'] = pd.to_datetime(clean_bikeshare['Merge Time']).dt.tz_convert('US/Eastern')


data_merged = pd.merge(clean_bikeshare, clean_weather, how = 'left', left_on = 'Merge Time', right_on = 'Date/Time')

# View DataFrame
data_merged.head()

,Trip Id,Start Time,End Time,Trip Duration,Start Station Id,Start Station Name,End Station Id,End Station Name,User Type,Merge Time,...,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Hmdx,Wind Chill,Weather
0,712441,2017-01-01 00:03:00-05:00,2017-01-01 00:08:00-05:00,274,7006,Bay St / College St (East Side),7021,Bay St / Albert St,Annual Member,2017-01-01 00:00:00-05:00,...,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN
1,712442,2017-01-01 00:03:00-05:00,2017-01-01 00:12:00-05:00,538,7046,Niagara St / Richmond St W,7147,King St W / Fraser Ave,Annual Member,2017-01-01 00:00:00-05:00,...,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN
2,712443,2017-01-01 00:05:00-05:00,2017-01-01 00:22:00-05:00,992,7048,Front St / Yonge St (Hockey Hall of Fame),7089,Church St / Wood St,Annual Member,2017-01-01 00:00:00-05:00,...,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN
3,712444,2017-01-01 00:09:00-05:00,2017-01-01 00:26:00-05:00,1005,7177,East Liberty St / Pirandello St,7202,Queen St W / York St (City Hall),Annual Member,2017-01-01 00:00:00-05:00,...,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN
4,712445,2017-01-01 00:14:00-05:00,2017-01-01 00:25:00-05:00,645,7203,Bathurst St/Queens Quay(Billy Bishop Airport),7010,King St W / Spadina Ave,Annual Member,2017-01-01 00:00:00-05:00,...,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN


<h2> Exporting Cleaned Data to .csv </h3>

In [ ]:
data_merged.to_csv('clean_data.csv')